<a href="https://colab.research.google.com/github/SDS-AAU/SDS-master/blob/master/M2/notebooks/network_analysis_application_bipartite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Load standardpackages
library(tidyverse) # Collection of all the good stuff like dplyr, ggplot2 ect.
library(magrittr) # For extra-piping operators (eg. %<>%)

library(tidygraph)
library(ggraph)
library(igraph)



In this session, you will learn:

1. What are alternative ways to create network structures.
2. What are different options to visualize networks and highlight properties.
3. How to analyse multi-modal networks.

# Types of networks

We up to now already talked about different ways how networks can be constructed. Up to now, we mainly focussed on:

* Interaction between entities
* Co-occurence

However, network analysis and modelling is also fully consistent with other structures, which are often a natural outcome of supervised or unsupervised ML exercises:

* Similarities
* Hirarchies (tree-structures)

## Similarity networks

* Since similarity is a relational property between entities, similarity matrices obviously can be modeled as a network. Lets illustrate that at the classican `mtcars` example.



In [ ]:
mtcars %>% head()


* We could first run a PCA to reduce the dimensionality of the numerical data.



In [ ]:
cars_pca <- mtcars[,c(1:7,10,11)] %>%
  drop_na() %>%
  prcomp(center = TRUE , scale = TRUE)


* Next, we could create a distance matrix (using the `dist()`) function.



In [ ]:
cars_dist <- cars_pca$x %>% dist(method = "euclidean")


La voila. Such a distance matrix representas a relational structure and can be modelled as a network.



In [ ]:
g <- cars_dist %>%
  as.matrix() %>%
  as_tbl_graph(directed = FALSE)

In [ ]:
g <- g %>% simplify() %>% as_tbl_graph()

In [ ]:
g


* Since the network is based on a distance matrix, we would like to reverse that to get edges representing similarity.
* Since similarity structures are usually fully connected networks, we probably also want to create some sparsity by deleting lower quantile edge weights.



In [ ]:
g <- g %E>%
  mutate(weight = max(weight) - weight) %>%
  filter(weight >= weight %>% quantile(0.75)) %N>%
  filter(!node_is_isolated()) %>%
  mutate(community = group_louvain(weights = weight) %>% factor())


Lets take a look!



In [ ]:
set.seed(1337)
g %>% ggraph(layout = "nicely") +
  geom_node_point(aes(col = community, size = centrality_degree(weights = weight))) +
  geom_edge_link(aes(width = weight), alpha = 0.25) +
  scale_edge_width(range = c(0.1, 2)) +
  geom_node_text(aes(label = name, filter = percent_rank(centrality_degree(weights = weight)) > 0.5), repel = TRUE) +
  theme_graph() +
  theme(legend.position = 'bottom')



## Hierarchy (tree) networks

Hirarchical structures are obviously also relational. The difference is, that the connectivity structure tends to be constraint to other levels.



In [ ]:
create_tree(20, 3) %>%
    mutate(leaf = node_is_leaf(), root = node_is_root()) %>%
    ggraph(layout = 'tree') +
    geom_edge_diagonal() +
    geom_node_point(aes(filter = leaf), colour = 'forestgreen', size = 10) +
    geom_node_point(aes(filter = root), colour = 'firebrick', size = 10) +
    theme_graph()


* In addition to real life examples such as organigrams, evolutionary trees etc., many ML models result in tree-structures (eg. decision trees).
* We will at our car example execute a hierarchical clustering, which leads to a tree structure (visualized in the dendogram).



In [ ]:
cars_hc <- cars_dist %>%
  hclust(method = "ward.D2")


Again, this structure can be directly transfered to a graph object.



In [ ]:
g <- cars_hc %>% as_tbl_graph()

In [ ]:
g

In [ ]:
g %>% ggraph(layout = 'dendrogram') +
  geom_edge_diagonal(aes(col = .N()$height[from])) +
  geom_node_point(aes(col =height)) +
  geom_node_text(aes(filter = leaf, label = label), angle=90, hjust=1, nudge_y=-0.1) +
  theme_graph() +
  theme(legend.position = 'none')
  ylim(-0.6, NA)



# Multi-Modal Networks

## Intuition

* Now its time to talk about an interesting type of networks, multi-modal. This means, a network has several "modes", meaning connects entities on different conceptual levels. T
* he most common one is a **2-mode** (or **bipartite**) network.
* Examples could be an Author $\rightarrow$ Paper, Inventor $\rightarrow$ Patent, Member $\rightarrow$ Club network.
* Here, the elements in the different modes represent different things.
* We can alalyse them in seperation (and sometimes we should), but often its helpful to "project"" them onto one mode.
* Here, we create a node in one mode by joint association with another mode.

![](https://sds-aau.github.io/SDS-master/00_media/networks_2mode.png){width=500px}
## Demonstartion

* While that sounds simple, it can be a very powerful technique, as I will demonstrate now.



In [ ]:
set.seed(1337)
g <- create_bipartite(4, 10, directed = TRUE, mode = "out") %E>%
  sample_n(15)

In [ ]:
people <- c('Jesper', 'Pernille', 'Morten', 'Lise', 'Christian', 'Mette', 'Casper', 'Dorte', 'Jacob', 'Helle')
places <- c('Yoga House', 'Crossfit', 'Jazz Club', 'Jomfru Anne Gade')

In [ ]:
g <- g %N>%
  mutate(name = c(places, people))

In [ ]:
g

In [ ]:
set.seed(1337)
p0 <- g %>% ggraph("bipartite") +
  geom_edge_link(alpha = 0.25) +
  geom_node_point(aes(col = type, size = centrality_degree(mode = 'all'))) +
  geom_node_text(aes(label = name), repel = TRUE) +
  theme_graph() +
  theme(legend.position = 'none') +
  labs(title = '2-mode network places-people')

p0


## Creating bipartite projections

### Within the fraph object

* Having a 2-mode network, we can use the `igraph` function `bipartite_projection` to create a 2x 1-mode network out of it.



In [ ]:
g_projected <- g %>% bipartite_projection()


* We now will have a 1-mode network between people as well as one between places.



In [ ]:
g_projected

In [ ]:
g_places <- g_projected[['proj1']] %>% as_tbl_graph(directed = FALSE)
g_people <- g_projected[['proj2']] %>% as_tbl_graph(directed = FALSE)


* Lets take a look:



In [ ]:
set.seed(1337)
library(patchwork)

p1 <- g_places %>% ggraph(layout = "nicely") +
  geom_node_point(aes(size = centrality_degree(weights = weight)), col = 'red') +
  geom_edge_link(aes(width = weight), alpha = 0.25) +
  scale_edge_width(range = c(0.1, 2)) +
  geom_node_text(aes(label = name), repel = TRUE) +
  theme_graph() +
  theme(legend.position = 'none') +
  labs(title = '1-mode network places')

p2 <- g_people %>% ggraph(layout = "nicely") +
  geom_node_point(aes(size = centrality_degree(weights = weight)), col = 'skyblue2') +
  geom_edge_link(aes(width = weight), alpha = 0.25) +
  scale_edge_width(range = c(0.1, 2)) +
  geom_node_text(aes(label = name), repel = TRUE) +
  theme_graph() +
  theme(legend.position = 'none') +
  labs(title = '1-mode network people')

p0 / (p1 + p2)


* Alright, but lets assume we have a 2-mode edgelist to start with... what possibilities do we have then?



In [ ]:
el_2m <- g %E>%
  mutate(from_name = .N()$name[from],
         to_name = .N()$name[to]) %>%
  as.tibble() %>%
  select(to_name, from_name) %>%
  rename(from = to_name, to = from_name)

In [ ]:
el_2m


* Such an edgelist could obviously be loaded into a graph object the usual way.
* We just have to assign types then



In [ ]:
g_2m <- el_2m %>% as_tbl_graph(directed = TRUE)  %N>%
  mutate(type = name %in% (el_2m %>% pull(from)))



### Via matrix nultiplication

* We can also do the projection outside of the graph and first create a 2-mode matrix.
* This can easily be done by crosstabulating the edgelist.



In [ ]:
mat_2m <- el_2m %>%
  table() %>%
  as.matrix()

In [ ]:
mat_2m


* Again, sparse matrices are usually more efficient.



In [ ]:
library(Matrix)
mat_2m %<>% Matrix(sparse = TRUE)

mat_2m



* Matrix algebra can help to do the 1-mode projection directly in the matrix
* Taking the dotproduct of the matrix and its transposed form will result in the 1-mode projection of mode 1 (`m %*% t(m)`)



In [ ]:
mat_people <- mat_2m %*% t(mat_2m)

In [ ]:
mat_people


* Taking the dotproduct of the transposed matrix and its original form will result in the 1-mode projection of mode 1 (`t(m) %*% m`)



In [ ]:
mat_places <- t(mat_2m) %*% mat_2m

In [ ]:
mat_places


* Note this is still very inefficient, since the matrix is first created in full, and then transformed to a sparse one.
* Directly starting with a sparse matrix makes the process much more efficient
* That makes a huge difference for large graphs
* I here provide you an efficient function to use



In [ ]:
## Helper function
el_to_sparse_matrix <- function(data, # the edgelist
                                mode_1, # which variable indicates mode 1
                                mode_2, # which variable indicates mode 2
                                projection = 'none' # If a pojection should be done, possible is 'none', 'mode1', 'mode2'
                                ){

  # Define inputs
  i_input <- data %>% pull({{mode_1}})
  j_input <- data %>% pull({{mode_2}})

  require(Matrix)
  mat <- spMatrix(nrow = i_input %>% n_distinct(),
                  ncol = j_input %>% n_distinct(),
                  i = i_input %>% factor() %>% as.numeric(),
                  j = j_input %>% factor() %>% as.numeric(),
                  x = rep(1, i_input %>% length() ) )

  row.names(mat) <- i_input %>% factor() %>% levels()
  colnames(mat) <- j_input %>% factor() %>% levels()

  # Projection if necessary
  if(projection == 'mode1'){mat %<>% tcrossprod()}
  if(projection == 'mode2'){mat %<>% crossprod()}

  return(mat)
}

In [ ]:
mat_people <- el_2m %>% el_to_sparse_matrix(from, to, projection = 'mode1')
mat_places <- el_2m %>% el_to_sparse_matrix(from, to, projection = 'mode2')

In [ ]:
mat_people
mat_places



### Via Joins

* FInally, the easiest way



In [ ]:
el_people <- el_2m %>% left_join(el_2m, by = 'to') %>%
  select(-to) %>%
  rename(from = from.x, to = from.y) %>%
  filter(from != to) %>%
  count(from, to, name = 'weight')

In [ ]:
el_people

In [ ]:
el_places <- el_2m %>% left_join(el_2m, by = 'from') %>%
  select(-from) %>%
  rename(from = to.x, to = to.y) %>%
  filter(from != to) %>%
  count(from, to, name = 'weight')

In [ ]:
el_places



# Case study: Bibliographic networks

## Basics

Lets talk about bibliographic networks. In short, that are networks between documents which cite each others. That can be (commonly) academic publications, but also patents or policy reports. Conceptually, we can see them as 2 mode networks, between articles and their reference. That helps us to apply some interesting metrics, such as:

* direct citations
* Bibliographic coupling
* Co--citations

Interestingly, different projections of this 2-mode network give the whole resulting 1-mode network a different meaning.

![](https://sds-aau.github.io/SDS-master/00_media/networks_biblio.png){width=500px}

* We will here do a brief bibliometric network analysis.
* While there exist specialized packages to do it more conveniently (eg. [bibliometrix](https://www.bibliometrix.org/)), we will for mximum clarity construct everything somewhat by hand.

* I will illustrate more in detail in the following. The example is based on some own work, where i here in  very simple way recreate some parts of the analysis.
* Rakas, M., & Hain, D. S. (2019). The state of innovation system research: What happens beneath the surface?. Research Policy, 45 (9). DOI: https://doi.org/10.1016/j.respol.2019.04.011


## The Data

* We will use bibliometrix data on articles from Scopus on recent publications containing the term `network analysis` in their title, abstract, or keywords.
* To do so, we first use the following search string: `TITLE-ABS-KEY ( "network analysis" )  AND  ( LIMIT-TO ( DOCTYPE ,  "ar" )  OR  LIMIT-TO ( DOCTYPE ,  "cp" ) )  AND  ( LIMIT-TO ( LANGUAGE ,  "English" ) )  AND  ( LIMIT-TO ( SRCTYPE ,  "j" )  OR  LIMIT-TO ( SRCTYPE ,  "p" ) )  AND  ( LIMIT-TO ( PUBYEAR ,  2021 )  OR  LIMIT-TO ( PUBYEAR ,  2020 )  OR  LIMIT-TO ( PUBYEAR ,  2019 )  OR  LIMIT-TO ( PUBYEAR ,  2018 )  OR  LIMIT-TO ( PUBYEAR ,  2017 )  OR  LIMIT-TO ( PUBYEAR ,  2016 )  OR  LIMIT-TO ( PUBYEAR ,  2015 ) ) `
* We sort the results by citations, and download the 2k most cited ones as `csv`. We select al possible fields to download.



In [ ]:
rm(list=ls())
data <- read_csv('https://github.com/SDS-AAU/SDS-master/raw/master/00_data/networks_bibliometrics/biblio_nw.csv')

In [ ]:
data %>%
  glimpse()


## Preprocessing



In [ ]:
data %<>%
  select(EID, Authors, `Author(s) ID`, Title, `Abbreviated Source Title`, Year, Affiliations, `Author Keywords`, `Cited by`, Abstract, References) %>%
  rename(author = Authors,
         author_id = `Author(s) ID`,
         title = Title,
         journal = `Abbreviated Source Title`,
         year = Year,
         affiliation = Affiliations,
         keywords = `Author Keywords`,
         citations = `Cited by`,
         Abstract = Abstract,
         references = References)

In [ ]:
data[2, 'author']

In [ ]:
data[2, 'author_id']

In [ ]:
data[2, 'affiliation']

In [ ]:
data[2, "keywords"]

In [ ]:
data[2, "references"]

In [ ]:
data %<>%
  mutate(author = author %>% str_split(', '),
         author_id = author_id %>% str_split(';'),
         affiliation = affiliation %>% str_split(';'),
         keywords = keywords %>% str_split('; '),
         references = references %>% str_split('; '))

In [ ]:
data %>% select(title, author, citations) %>%
  unnest(author) %>%
  head()

In [ ]:
data %>% select(title, author, citations) %>%
  unnest(author) %>%
  group_by(author) %>%
  summarise(n = n(),
            citations = citations %>% sum(na.rm = TRUE)) %>%
  arrange(desc(citations)) %>%
  head(10)

## Bipartite Network Generation

I will now transfer them to an article $\rightarrow$ reference edgelist.



In [ ]:
el_2m <- data %>% select(EID, references) %>% unnest(references)

In [ ]:
el_2m %>% head()


### Bibliographic Coupling

#### Create the graph

* We now need to create a projection of the 2-mode matrix to one of the modes.
* We could do that more efficiently via using the sparse matrix calculation I did before, but for maximum clarity lets do it via a join.
* We will first create the article to article network.



In [ ]:
el_bib <- el_2m %>% left_join(el_2m, by = 'references') %>%
  rename(from = EID.x,
         to = EID.y) %>%
  select(from, to) %>%
  filter(from != to) %>%
  count(from, to, name = 'weight')

In [ ]:
el_bib %>% head()

In [ ]:
el_bib %>%
  ggplot(aes(x = weight)) +
  geom_histogram()

In [ ]:
el_bib %<>%
  filter(weight >=2)

In [ ]:
el_bib %>%
  ggplot(aes(x = weight)) +
  geom_histogram()

* Ok, great, we now can create a graph



In [ ]:
g_bib <- el_bib %>% as_tbl_graph(directed = FALSE) %>%
  igraph::simplify() %>%
  as_tbl_graph(directed = FALSE)

In [ ]:
g_bib

In [ ]:
g_bib <- g_bib %N>%
  filter(!node_is_isolated()) %N>%
  filter(percent_rank(centrality_degree(weights = weight)) >= 0.5)



* A first obvious thing to do would be a community detection



In [ ]:
g_bib <- g_bib %N>%
  mutate(com = group_louvain(weights = weight))


* Now, we can also create community internal statistics, such as the within-community degree.
* To do so, we use the powerful `morph()` functions of `tidygraph`, which basically let you apply `group_by` style operations on graph structures, where graph calculations are execited on subgraphs.
* That let you e.g. contract nodes, work on the linegraph representation, split communities to separate graphs etc.



In [ ]:
g_bib <- g_bib %N>%
  morph(to_split, com) %>%
    mutate(cent_dgr_int = centrality_degree(weights = weight)) %>%
    mutate(com_center = cent_dgr_int == max(cent_dgr_int)) %>%
  unmorph()

In [ ]:
g_bib %N>%
  as_tibble() %>%
  count(com, sort = TRUE)


* Ok, seems like there are too many communities to do something meaningful.
* Lets restrict it to only only communities with more than 20 members.



In [ ]:
g_bib <- g_bib %N>%
  add_count(com, name = 'com_n') %>%
  mutate(com = ifelse(com_n >= 20, com, NA) ) %>%
  select(-com_n)

In [ ]:
g_bib <- g_bib %N>%
  left_join(data %>% select(EID, title, journal, year, citations), by = c('name' = 'EID')) %>%
  mutate(title = title %>% str_trunc(30))



* Lets take a look at the network.



In [ ]:
set.seed(1337)
g_bib %>%
  ggraph(layout = 'graphopt') +
  geom_edge_link(aes(width = weight,
                     color = .N()$com[from] %>% as.factor()), # Notice that
                alpha = 0.5,
                show.legend = FALSE) +
      scale_edge_width(range = c(0.5, 2)) +
  geom_node_point(aes(color = com %>% as.factor(),
                      size = centrality_degree(weight = weight),
                      alpha = citations)) +
  geom_node_text(aes(label = title, filter = com_center == TRUE), repel = TRUE) +
  theme_graph()   +
  theme(legend.position = 'bottom') +
  labs(title = 'Bibliographic Coupling Network',
       subtitle = 'Network Analysis 2015-2020',
       color = 'Community',
       size = 'Degree',
       alpha = 'Citations')

In [ ]:
data %>%
  select(EID, author, year, title, journal, citations) %>%
  inner_join(g_bib %N>% as_tibble() %>% select(name, com, cent_dgr_int), by = c('EID' = 'name')) %>%
  group_by(com) %>%
    arrange(desc(cent_dgr_int)) %>%
    slice(1:10) %>%
  ungroup() %>%
  select(com, title, cent_dgr_int, citations) %>%
  mutate(title = title %>% str_trunc(75))


## Co-Citation Analysis

* We can now do exactly the same for the reference (=co-citation) network

#### Create the graph



In [ ]:
el_cit <- el_2m %>% left_join(el_2m, by = 'EID') %>%
  rename(from = references.x,
         to = references.y) %>%
  select(from, to) %>%
  filter(from != to) %>%
  count(from, to, name = 'weight')

In [ ]:
el_cit %<>%
  filter(weight >=3,
         !str_detect(from, '\\:\\, '),
         !str_detect(to, '\\:\\, '),
         str_length(from) > 50,
         str_length(to) > 50
         )

In [ ]:
el_cit %>%
  ggplot(aes(x = weight)) +
  geom_histogram()

In [ ]:
g_cit <- el_cit %>% as_tbl_graph(directed = FALSE) %>%
  igraph::simplify() %>%
  as_tbl_graph(directed = FALSE)  %N>%
  filter(!node_is_isolated()) %N>%
  filter(percent_rank(centrality_degree(weights = weight)) >= 0.5) %N>%
  mutate(com = group_louvain(weights = weight)) %N>%
  morph(to_split, com) %>%
    mutate(cent_dgr_int = centrality_degree(weights = weight)) %>%
    mutate(com_center = cent_dgr_int == max(cent_dgr_int)) %>%
  unmorph()

In [ ]:
g_cit %N>%
  as_tibble() %>%
  count(com, sort = TRUE)


* Ok, seems like there are too many communities to do something meaningful.
* Lets restrict it to only only communities with more than 20 members.



In [ ]:
g_cit <- g_cit %N>%
  add_count(com, name = 'com_n') %>%
  mutate(com = ifelse(com_n >= 10, com, NA) ) %>%
  select(-com_n)


Lets take a look at the network.



In [ ]:
set.seed(1337)
g_cit %N>%
  mutate(name = name %>% str_trunc(75)) %>%
  ggraph(layout = 'graphopt') +
  geom_edge_link(aes(width = weight,
                     color = .N()$com[from] %>% as.factor()), # Notice that
                alpha = 0.5,
                show.legend = FALSE) +
      scale_edge_width(range = c(0.5, 2)) +
  geom_node_point(aes(color = com %>% as.factor(),
                      size = centrality_degree(weight = weight),
                      alpha = cent_dgr_int)) +
  geom_node_text(aes(label = name, filter = com_center == TRUE & percent_rank(cent_dgr_int) > 0.80 ), repel = TRUE) +
  theme_graph()   +
  theme(legend.position = 'bottom') +
  labs(title = 'Bibliographic Coupling Network',
       subtitle = 'Network Analysis 2015-2020',
       color = 'Community',
       size = 'Degree',
       alpha = 'Citations')

In [ ]:
g_cit %N>%
  as_tibble() %>%
  group_by(com) %>%
    arrange(desc(cent_dgr_int)) %>%
    slice(1:10) %>%
  ungroup() %>%
  mutate(name = name %>% str_trunc(75)) %>%
  select(com, name, cent_dgr_int)



## Joint analysis



In [ ]:
el_joint <- el_2m %>%
  inner_join(g_bib %N>% as_tibble() %>% select(name, com) %>% drop_na(), by = c('EID' = 'name')) %>%
  inner_join(g_cit %N>% as_tibble() %>% select(name, com) %>% drop_na(), by = c('references' = 'name')) %>%
  rename(from = com.x, to = com.y) %>%
  count(from, to, name = 'weight')

In [ ]:
el_joint

In [ ]:
el_joint %<>%
  mutate(from = paste('bib', from, sep = '_'),
         to = paste('cit', to, sep = '_'))

In [ ]:
g_joint <- el_joint %>% as_tbl_graph(directed = TRUE)

In [ ]:
g_joint <- g_joint %N>%
  mutate(type = name %>% str_detect('bib'))

In [ ]:
g_joint %>% ggraph("bipartite") +
  geom_edge_link(alpha = 0.25) +
  geom_node_point(aes(col = type, size = centrality_degree(mode = 'all'))) +
  geom_node_text(aes(label = name), repel = TRUE) +
  theme_graph() +
  theme(legend.position = 'none')



# Endnotes

### Complementary exercises

Please do **Exercise 1** in the corresponding section on `Github`. This time you are about to do your own bibliographic analysis!

### References

Paper mentioned in the text

* Rakas, M., & Hain, D. S. (2019). The state of innovation system research: What happens beneath the surface?. Research Policy, 45 (9). DOI: https://doi.org/10.1016/j.respol.2019.04.011

Other own work dealing with 2-mode networks

* Hain, Daniel S., and Roman Jurowetzki. "Incremental by Design? On the Role of Incumbents in Technology Niches." In Foundations of Economic Change, pp. 299-332. Springer, Cham, 2017.
* Hain, D., Buchmann, T., Kudic, M., & Müller, M. (2018). Endogenous dynamics of innovation networks in the German automotive industry: analysing structural network evolution using a stochastic actor-oriented approach. International Journal of Computational Economics and Econometrics, 8(3-4), 325-344.
* Jurowetzki, Roman, and Daniel S. Hain. "Mapping the (r-) evolution of technological fields–a semantic network approach." Social Informatics, pp. 359-383. Springer, Cham, 2014.

### Packages & Ecosystem

You can find more info about:

* `tidygraph` [here](https://tidygraph.data-imaginist.com/)
* `ggraph` [here](https://ggraph.data-imaginist.com/)
* `bibliometrix` [here](http://www.bibliometrix.org/)

### Other Sources

* An example notebook where I use the ideas presented hee for a simple analysis of technological relatedness can be found [here](https://rawcdn.githack.com/daniel-hain/SDC_IM/9bf8683ffeea703e50e5506ac0eb3dd7544621c3/S3_2_Economic_complexity.html)


### Session info


In [ ]:
sessionInfo()